In [ ]:
dataset_filename = "dataset_lemmatized_full.json"
dataset_path = f"/content/drive/MyDrive/diploma-llm/data/lda/{dataset_filename}"
limit_documents = -1

# Training parameters
num_topics = 1000
passes = 15

model_name = "lda-lg-phrases.model"
model_dir = "lda-lg-phrases"
model_path = f"/content/drive/MyDrive/diploma-llm/models/lda/{model_dir}/{model_name}"

dictionary_filename = f"{model_name}.id2word"
dictionary_path = f"/content/drive/MyDrive/diploma-llm/models/lda/{model_dir}/{dictionary_filename}"

In [ ]:
!pip install gensim

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open(dataset_path, "r") as source:
  documents = json.load(source)

In [ ]:
if limit_documents > 0:
  documents = documents[0:limit_documents]

In [ ]:
from gensim.parsing.preprocessing import preprocess_string

# Preprocessing the text data
processed_docs = [preprocess_string(doc) for doc in documents]

In [ ]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(processed_docs, min_count=20, threshold=1.0)
for idx in range(len(processed_docs)):
    for token in bigram[processed_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            processed_docs[idx].append(token)

In [ ]:
from gensim import corpora

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(processed_docs)

# Create a corpus from the dictionary and processed documents
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
from gensim.models.ldamodel import LdaModel

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=passes)

In [ ]:
lda_model.save(model_path)

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string


lda_model = LdaModel.load(model_path)
dictionary = corpora.Dictionary.load(dictionary_path)

In [ ]:
# Print the Keyword in the 10 topics
topics = lda_model.print_topics()
for topic in topics:
    print(topic)

(46, '0.033*"значення" + 0.025*"функція" + 0.010*"коефіцієнт" + 0.010*"формула" + 0.010*"рівняння" + 0.009*"наступний" + 0.009*"випадок" + 0.009*"система" + 0.008*"параметр" + 0.008*"матриця"')
(78, '0.014*"соціальний" + 0.009*"людина" + 0.008*"політичний" + 0.008*"суспільство" + 0.008*"конфлікт" + 0.005*"життя" + 0.005*"група" + 0.005*"дослідження" + 0.005*"рік" + 0.004*"особа"')
(10, '0.037*"напруга" + 0.021*"схема" + 0.018*"живлення" + 0.015*"струму" + 0.015*"перетворювач" + 0.012*"струм" + 0.010*"мікроконтролер" + 0.009*"модуль" + 0.008*"вихідний" + 0.008*"роботи"')
(17, '0.051*"матеріал" + 0.017*"властивість" + 0.014*"міцність" + 0.011*"механічний" + 0.011*"деформація" + 0.010*"високий" + 0.009*"поверхня" + 0.009*"процес" + 0.009*"напруження" + 0.007*"порошок"')
(86, '0.171*"таблиця" + 0.063*"рисунок" + 0.040*"кількість" + 0.033*"результат" + 0.021*"навести" + 0.019*"час" + 0.018*"наведений" + 0.017*"показник" + 0.016*"продовження" + 0.014*"продовження_таблиця"')
(30, '0.044*"сист

In [ ]:
bow_vector = dictionary.doc2bow(preprocess_string(documents[2567]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: tup[1], reverse=True):
    print("Score: {}\t Topic({}): {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.49297866225242615	 Topic(36): 0.017*"країна" + 0.014*"розвиток" + 0.008*"міжнародний" + 0.008*"економічний" + 0.007*"виробництво"
Score: 0.21226955950260162	 Topic(88): 0.044*"підприємство" + 0.022*"діяльність" + 0.020*"дослідження" + 0.019*"конкурентоспроможність" + 0.019*"економічний"
Score: 0.09733373671770096	 Topic(96): 0.028*"могти" + 0.011*"метод" + 0.009*"рішення" + 0.009*"певний" + 0.007*"використовуватися"
Score: 0.041753631085157394	 Topic(42): 0.052*"ризик" + 0.030*"фінансовий" + 0.018*"інвестиційний" + 0.012*"ринку" + 0.012*"ціна"
Score: 0.021744702011346817	 Topic(78): 0.014*"соціальний" + 0.009*"людина" + 0.008*"політичний" + 0.008*"суспільство" + 0.008*"конфлікт"
Score: 0.020809054374694824	 Topic(61): 0.046*"витрата" + 0.041*"грн" + 0.030*"проект" + 0.015*"вартість" + 0.013*"реалізація"
Score: 0.016223901882767677	 Topic(55): 0.025*"управління" + 0.021*"підприємство" + 0.015*"персонал" + 0.012*"система" + 0.012*"працівник"
Score: 0.016007140278816223	 Topic(99

In [ ]:
doc_bow = dictionary.doc2bow(preprocess_string(documents[2567]))  # Convert document to bag-of-words format
doc_topics = lda_model.get_document_topics(doc_bow)
print(doc_topics)

[(36, 0.49288377), (42, 0.041725658), (46, 0.0155226225), (55, 0.016192986), (61, 0.020706175), (78, 0.021755178), (88, 0.21230836), (96, 0.09739791), (99, 0.015961643)]


In [ ]:
document_to_check = """Вступ    Актуальність Існує безліч способів вирішення проблем, пов’язаних з обробкою й аналізом часових рядів, але вважається, що існує мало готових програмних комплексів, які мають компактний функціонал та допомагають користувачам обирати важливі інструменти обробки та аналізу збірок статистичних даних. Практичне значення результатів досліджень  має  вагоме  значення  в економіці та  інших  галузях  науково-технічного розвитку. Застосування отриманих результатів забезпечить швидкий аналіз даних та безпосереднє управління аналізом статистичних інформації, розвиток та  вибороть деякого розуміння або дозволить підтвердити чи спростувати згадану гіпотезу [1, 6]. Спрощений використовуваний інтерфейс програмного продукту в найкращий спосіб і у короткий час надає можливості керувати послідовністю кроків обробки та аналізу вибраної сукупності даних часових рядів. Чисельні експерименти були проведені на основі користування 3 базами даних для певного алгоритму отримання залежностей взаємодій серед компонент часового ряду. Предмет дослідження Предметом дослідження є аналіз, розробка та перевірка працездатності та ефективності застосунку при керуванні послідовністю кроків обробки та аналізу вибраної сукупності статистичних даних. Метою роботи є розробка програмного продукту, який мав би застосовуватись для організації та управління процесом аналізу часових рядів. В основу принципової можливості входить забезпечити дисципліноване тестування, зниження ймовірностей помилкових ухилення результатів аналізу, позначити адекватну оцінку до точності опису явища [1]. Метою роботи є опис статистичних даних та основних характеристик часових рядів, яким було вироблено програмне забезпечення для вибору технології аналізу та виділення важливих аспектів застосування моделей до реальних задач аналізу. """

In [ ]:
doc_bow = dictionary.doc2bow(preprocess_string(document_to_check))  # Convert document to bag-of-words format
doc_topics = lda_model.get_document_topics(doc_bow)

In [ ]:
for index, score in sorted(lda_model[doc_bow], key=lambda tup: tup[1], reverse=True):
    print("Score: {}\t Topic({}): {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.19015416502952576	 Topic(23): 0.023*"система" + 0.022*"аналіз" + 0.021*"дослідження" + 0.019*"роботи" + 0.016*"результат"
Score: 0.18201705813407898	 Topic(60): 0.074*"даних" + 0.060*"система" + 0.030*"база" + 0.018*"користувач" + 0.018*"база_даних"
Score: 0.18078014254570007	 Topic(34): 0.041*"даних" + 0.038*"модель" + 0.035*"навчання" + 0.018*"набір" + 0.018*"класифікація"
Score: 0.17317400872707367	 Topic(96): 0.028*"могти" + 0.011*"метод" + 0.009*"рішення" + 0.009*"певний" + 0.007*"використовуватися"
Score: 0.07233396172523499	 Topic(28): 0.017*"розробка" + 0.017*"мова" + 0.015*"додаток" + 0.012*"програмування" + 0.012*"дозволяти"
Score: 0.06629051268100739	 Topic(99): 0.071*"модель" + 0.026*"прогнозування" + 0.018*"даних" + 0.018*"аналіз" + 0.017*"значення"
Score: 0.049855660647153854	 Topic(41): 0.027*"логістичний" + 0.023*"витрата" + 0.019*"система" + 0.018*"процес" + 0.015*"управління"
Score: 0.026697669178247452	 Topic(78): 0.014*"соціальний" + 0.009*"людина" + 0.008*

In [ ]:
# expected_keywords = ['програма,' 'часовий_ряд', 'статистика']
expected_keywords = ['програма,' 'часовий_ряд', 'статистика', 'паралельне_обчислення', 'керівництво_користувач', 'програмування', 'додаток', 'розробка']
relevant_topics = {}

# First, map keywords to topics
for keyword in expected_keywords:
    for topic_no, words in lda_model.show_topics(formatted=False, num_topics=num_topics):
        for word, prob in words:
            if word == keyword:
                if topic_no in relevant_topics:
                    relevant_topics[topic_no] += prob  # increase relevance score
                else:
                    relevant_topics[topic_no] = prob

# Now, calculate the correspondence score
doc_score = 0
for topic_id, weight in doc_topics:
    if topic_id in relevant_topics:
        doc_score += weight * relevant_topics[topic_id]

print("Document Score:", doc_score)

Document Score: 0.005576279712840915


In [ ]:
from gensim.models import CoherenceModel
import gensim

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3, passes=3):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=passes)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_docs, start=5, limit=80, step=10, passes=10)

In [ ]:
import matplotlib.pyplot as plt
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'coherence_values' is not defined

In [ ]:
keywords = ['програма', 'часовий_ряд', 'статистика', 'паралельні обчислення']

In [ ]:
def topic_relevance(lda_model, keywords, num_topics, topn=50):
    keyword_set = set(keywords)
    topic_scores = []

    for topic_id in range(num_topics):
        topic_words = dict(lda_model.show_topic(topic_id, topn=topn))
        common_words = keyword_set.intersection(set(topic_words.keys()))
        score = sum(topic_words[word] for word in common_words)
        topic_scores.append((topic_id, score, common_words))

    return sorted(topic_scores, key=lambda x: x[1], reverse=True)

num_topics = lda_model.num_topics
relevant_topics = topic_relevance(lda_model, keywords, num_topics)

In [ ]:
print(relevant_topics)

[(3, 0.016223862767219543, {'програма'}), (50, 0.01273330207914114, {'програма'}), (28, 0.010666023008525372, {'програма'}), (95, 0.007328697480261326, {'програма'}), (39, 0.007178687024861574, {'програма'}), (2, 0.006976227276027203, {'програма'}), (53, 0.006697325501590967, {'програма'}), (85, 0.006395218428224325, {'програма'}), (69, 0.006077435798943043, {'програма'}), (52, 0.004454662092030048, {'програма'}), (38, 0.004365948494523764, {'програма'}), (58, 0.004351306706666946, {'програма'}), (99, 0.003657858818769455, {'часовий_ряд'}), (55, 0.0030612272676080465, {'програма'}), (0, 0, set()), (1, 0, set()), (4, 0, set()), (5, 0, set()), (6, 0, set()), (7, 0, set()), (8, 0, set()), (9, 0, set()), (10, 0, set()), (11, 0, set()), (12, 0, set()), (13, 0, set()), (14, 0, set()), (15, 0, set()), (16, 0, set()), (17, 0, set()), (18, 0, set()), (19, 0, set()), (20, 0, set()), (21, 0, set()), (22, 0, set()), (23, 0, set()), (24, 0, set()), (25, 0, set()), (26, 0, set()), (27, 0, set()), (2

In [ ]:
threshold = 0.01  # Example threshold
selected_topics = [topic for topic in relevant_topics if topic[1] > threshold]

print("Selected Topics:")
for topic in selected_topics:
    print(f"Topic ID: {topic[0]}, Score: {topic[1]}, Common Keywords: {topic[2]}")

Selected Topics:
Topic ID: 3, Score: 0.016223862767219543, Common Keywords: {'програма'}
Topic ID: 50, Score: 0.01273330207914114, Common Keywords: {'програма'}
Topic ID: 28, Score: 0.010666023008525372, Common Keywords: {'програма'}


In [ ]:
def extend_keywords_with_topics(lda_model, selected_topics, topn=5):
    extended_keywords = set()
    for topic_id, _, _ in selected_topics:
        topic_words = lda_model.show_topic(topic_id, topn=topn)
        extended_keywords.update([word for word, _ in topic_words])
    return extended_keywords

new_keywords = extend_keywords_with_topics(lda_model, selected_topics)
print("Extended Keywords:", new_keywords)

Extended Keywords: {'гучномовець', 'файл', 'кооперація', 'керівництво_користувач', 'керівництво', 'рисунок', 'вікно', 'концертний', 'програмування', 'мова', 'кнопка', 'додаток', 'дозволяти', 'розробка', 'програма'}
